In [52]:
import pandas as pd
import glob
import os

## Elaborate and confront dataframes from MPNN and ESM rank

### Load paths and fastas

In [53]:
#Path to the folders cointaining FASTA
FOLDER_GENERAL_PATH = "~/LigandMPNN/outputs/ARSB_pred/ARSB_*"

# Name of fasta generated by LigandMPNN
FASTA = 'ARSB_gapless.fa'

# MPNN raw filename
MPNN_RAW = 'mutation_list_ARSB_raw.csv'

#ESM RANK raw filename
ESM_RANK_RAW = '/home/tigem/a.valente/Desktop/superenzyme_project/ARSB/ARSB_HUMAN.csv'

# ESM RANK cooked filename
ESM_RANK_COOKED = 'Desktop/ARSB_HUMAN_cooked.csv'

# Final output filename
OUTPUT_FILE = '~/Desktop/mutation_list_ready_ARSB.csv'

#Wildtype sequence
WT ='SRPPHLVFLLADDLGWNDVGFHGSRIRTPHLDALAAGGVLLDNYYTQPLATPSRSQLLTGRYQIRTGLQHQIIWPCQPSCVPLDEKLLPQLLKEAGYTTHMVGKWHLGMYRKECLPTRRGFDTYFGYLLGSEDYYSHERCTLIDALNVTRCALDFRDGEEVATGYKNMYSTNIFTKRAIALITNHPPEKPLFLYLALQSVHEPLQVPEEYLKPYDFIQDKNRHHYAGMVSLMDEAVGNVTAALKSSGLWNNTVFIFSTDNGGQTLAGGNNWPLRGRKWSLWEGGVRGVGFVASPLLKQKGVKNRELIHISDWLPTLVKLARGHTNGTKPLDGFDVWKTISEGSPSPRIELLHNIDPNFVDSSPCPRNSMAPAKDDSSLPEYSAFNTSVHAAIRHGNWKLLTGYPGCGYWFPPPSQYNVSEIPSSDPPTKTLWLFDIDRDPEERHDLSREYPHIVTKLLSRLQFYHKHSVPVYFPAQDPRCDPKATGVWGPWM'

### Define functions for elaboration

In [54]:
def extract_best_seq(folder, fasta_name):
    """
    Extracts the best sequence from a fasta file in the given folder based on the highest score.

    Args:
        folder (str): Path to the folder containing the fasta file.

    Returns:
        dict: A dictionary with the mutant name as the key and a tuple (sequence id, score) as the value.
    """
    mutant = folder.split("/")[-1]
    dict_seqs = {mutant: ()}
    score = 0.0

    with open(f"{folder}/seqs/{fasta_name}", "r") as fi:
        for line in fi:
            if line.startswith(">"):
                # Check if the header contains "overall_confidence"
                if "overall_confidence" in line:
                    seq_rec = float(line.split(",")[-1].split("=")[1])
                    # Skip if confidence is exactly 1.0, meaning is not a mutant
                    if seq_rec == 1.0000:
                        continue
                    else:
                        # Extract the score from the header (5th comma-separated field)
                        new_score = float(line.split(",")[4].split("=")[1])
                        #print(new_score)

                        # If this score is higher than the current best, update the dictionary and score
                        if new_score > score:
                            dict_seqs[mutant] = (line.split(",")[1], line.split(",")[4].split("=")[1])
                            score = new_score

    return dict_seqs

In [55]:
def extract_seq(folder, id, fasta_name):
    # Open the fasta file for reading
    with open(f"{folder}/seqs/{fasta_name}", "r") as fi:
        lines = fi.readlines()
        # Iterate through each line to find the header with the given id
        for line in lines:
            if line.startswith(">") and id in line:
                # The sequence is expected to be the line immediately after the header
                seq = lines[lines.index(line) + 1]
                break
    return seq

In [56]:
def compare_seq(WT, mutant, pos):
    # Compare the amino acid at the given position (1-based index) between WT and mutant
    if mutant[pos-1] == WT[pos-1]:
        print(f"WT: {WT[pos-1]}, Mutant: {mutant[pos-1]} same")
    else:
        print(f"WT: {WT[pos-1]}, Mutant: {mutant[pos-1]} different")

    # Return the amino acids at the specified position for WT and mutant
    return WT[pos-1], mutant[pos-1]

In [57]:
# Create a dictionary where the key is the folder/mutant and the value is a tuple of (sequence id, score). For each mutant only the best sequence is kept.

dict_list = []

for folder in glob.glob(os.path.expanduser(FOLDER_GENERAL_PATH)):
    #print(folder)
    dict_list.append(extract_best_seq(folder, FASTA))

In [58]:
#The dictionary is empty if LigandMPNN produced only WT-like sequences
dict_list

[{'ARSB_A102': ()},
 {'ARSB_A168': ()},
 {'ARSB_A212': (' id=10', '0.4473')},
 {'ARSB_A525': ()},
 {'ARSB_A474': ()},
 {'ARSB_A268': ()},
 {'ARSB_A487': ()},
 {'ARSB_A171': (' id=1', '0.3865')},
 {'ARSB_A520': ()},
 {'ARSB_A481': ()},
 {'ARSB_A460': (' id=8', '0.1319')},
 {'ARSB_A182': (' id=1', '0.3642')},
 {'ARSB_A64': ()},
 {'ARSB_A290': ()},
 {'ARSB_A159': (' id=4', '0.5436')},
 {'ARSB_A311': ()},
 {'ARSB_A149': ()},
 {'ARSB_A315': ()},
 {'ARSB_A249': ()},
 {'ARSB_A273': ()},
 {'ARSB_A328': (' id=1', '0.2061')},
 {'ARSB_A414': (' id=7', '0.1420')},
 {'ARSB_A56': ()},
 {'ARSB_A395': ()},
 {'ARSB_A129': ()},
 {'ARSB_A217': (' id=10', '0.2377')},
 {'ARSB_A199': (' id=7', '0.5154')},
 {'ARSB_A110': (' id=13', '0.1696')},
 {'ARSB_A47': ()},
 {'ARSB_A455': (' id=7', '0.5217')},
 {'ARSB_A391': (' id=1', '0.3288')},
 {'ARSB_A114': ()},
 {'ARSB_A200': ()},
 {'ARSB_A119': ()},
 {'ARSB_A316': ()},
 {'ARSB_A253': (' id=4', '0.3250')},
 {'ARSB_A475': (' id=11', '0.4654')},
 {'ARSB_A78': (' id=1

In [59]:
# Filter out dictionaries from dict_list where the value tuple is empty
dict_list = [d for d in dict_list if list(d.values())[0]]

# Sort the filtered list of dictionaries by the score (second element of the value tuple) in descending order
dict_list_sorted = sorted(dict_list, key=lambda d: float(list(d.values())[0][1]), reverse=True)

# Display the sorted list
dict_list_sorted

[{'ARSB_A411': (' id=10', '0.9199')},
 {'ARSB_A229': (' id=13', '0.8998')},
 {'ARSB_A71': (' id=7', '0.8674')},
 {'ARSB_A430': (' id=4', '0.8394')},
 {'ARSB_A122': (' id=7', '0.8320')},
 {'ARSB_A453': (' id=1', '0.7846')},
 {'ARSB_A107': (' id=4', '0.7845')},
 {'ARSB_A263': (' id=10', '0.7824')},
 {'ARSB_A150': (' id=13', '0.7609')},
 {'ARSB_A61': (' id=7', '0.7383')},
 {'ARSB_A287': (' id=13', '0.7250')},
 {'ARSB_A399': (' id=7', '0.7194')},
 {'ARSB_A222': (' id=7', '0.7170')},
 {'ARSB_A330': (' id=4', '0.7104')},
 {'ARSB_A240': (' id=13', '0.6950')},
 {'ARSB_A464': (' id=13', '0.6834')},
 {'ARSB_A429': (' id=7', '0.6780')},
 {'ARSB_A317': (' id=1', '0.6421')},
 {'ARSB_A448': (' id=1', '0.6405')},
 {'ARSB_A305': (' id=1', '0.6370')},
 {'ARSB_A441': (' id=13', '0.6306')},
 {'ARSB_A461': (' id=10', '0.6251')},
 {'ARSB_A198': (' id=1', '0.6204')},
 {'ARSB_A339': (' id=7', '0.6141')},
 {'ARSB_A85': (' id=13', '0.6120')},
 {'ARSB_A214': (' id=10', '0.6053')},
 {'ARSB_A63': (' id=1', '0.591

### MPNN CSV creation

In [60]:
# # Create a new CSV with ProteinMPNN results
# # This script will strongly depend on the name format you chose for the mutant folders

OFFSET = 41  # Offset to adjust the position to match the 1-based index

with open( MPNN_RAW, "w") as fo:
     # Write the header line
     fo.write("Mutant,WT_pos,Position,Mut_pos,Score\n")
     # Iterate through each mutant dictionary in the sorted list
     for mutant_dict in dict_list_sorted:
         # Skip the default entry if present
         if list(mutant_dict.keys())[0] != "4KL5_A1C_def":
             mutant_id = list(mutant_dict.keys())[0]
             # Extract the mutation position from the mutant ID
             try:
                 raw_position = int(mutant_id.split("_")[-1][1:])
                # Adjust the position to match the 1-based index
                 position = raw_position - OFFSET
                 # Get the sequence ID and score for this mutant
                 seq_id = list(mutant_dict.values())[0][0]
                 score = list(mutant_dict.values())[0][1]
                 # Extract the mutant sequence using the sequence ID
                 mutant_seq = extract_seq(f"LigandMPNN/outputs/ARSB_pred/{mutant_id}", seq_id, FASTA)
                 # Compare the WT and mutant amino acids at the mutation position
                 WT_pos, mut_pos = compare_seq(WT, mutant_seq, position)
                
                 # Print mutation details for debugging
                 print(mutant_id, WT_pos, position, mut_pos, seq_id, score)
                 # Write the mutation information to the file
                 fo.write(f"{mutant_id},{WT_pos},{position},{mut_pos},{score}\n")
             except ValueError:
                 print(f"Skipping mutant {mutant_id} due to Error in parsing position")
                 continue

WT: A, Mutant: P different
ARSB_A411 A 370 P  id=10 0.9199
WT: E, Mutant: G different
ARSB_A229 E 188 G  id=13 0.8998
WT: H, Mutant: N different
ARSB_A71 H 30 N  id=7 0.8674
WT: H, Mutant: T different
ARSB_A430 H 389 T  id=4 0.8394
WT: V, Mutant: I different
ARSB_A122 V 81 I  id=7 0.8320
WT: P, Mutant: D different
ARSB_A453 P 412 D  id=1 0.7846
WT: T, Mutant: L different
ARSB_A107 T 66 L  id=4 0.7845
WT: R, Mutant: A different
ARSB_A263 R 222 A  id=10 0.7824
WT: M, Mutant: W different
ARSB_A150 M 109 W  id=13 0.7609
WT: G, Mutant: S different
ARSB_A61 G 20 S  id=7 0.7383
WT: S, Mutant: A different
ARSB_A287 S 246 A  id=13 0.7250
WT: F, Mutant: Y different
ARSB_A399 F 358 Y  id=7 0.7194
WT: L, Mutant: I different
ARSB_A222 L 181 I  id=7 0.7170
WT: G, Mutant: A different
ARSB_A330 G 289 A  id=4 0.7104
WT: S, Mutant: D different
ARSB_A240 S 199 D  id=13 0.6950
WT: S, Mutant: T different
ARSB_A464 S 423 T  id=13 0.6834
WT: V, Mutant: I different
ARSB_A429 V 388 I  id=7 0.6780
WT: R, Mutant

## Lookup with ESM_rank file

In [61]:
# Load ESM_rank file (called "riccardo" because i say so)

riccardo = pd.read_csv(ESM_RANK_RAW)

riccardo

,ids,prediction
0,F233P,-1.050642
1,W146P,-0.851135
2,W353P,-0.827151
3,F331P,-0.819417
4,C155P,-0.809750
...,...,...
10122,R407K,0.873944
10123,R180K,0.876465
10124,R479K,0.879484
10125,R345K,0.896970


In [62]:
# Add columns for position, wildtype, and mutation to the ESM_rank dataframe

riccardo['pos'] = [ pos for pos in riccardo['ids'].str.extract('([A-Z])(\d+)')[1].astype(int)]
riccardo['wt'] = [ wt for wt in riccardo['ids'].str.extract('([A-Z])(\d+)')[0]]
riccardo['mut'] = [mut for mut in riccardo['ids'].str.extract('(\d+)([A-Z])')[1]]

In [63]:
# sorting by position, just for aesthetic

riccardo.sort_values(by='pos', ascending=True, inplace=True)

In [64]:
ESM_OFFSET = 41  # Offset to exclude positions not considered in MPNN analysis

# Remove positions not considered in MPNN analysis (in this case, positions 1-31)
riccardo = riccardo[riccardo['pos'] > ESM_OFFSET]

In [65]:
# now we have the cooked dataframe with the columns we want. Let's save it.

riccardo.to_csv(ESM_RANK_COOKED , index=False)

In [66]:
# show it off, bitch
riccardo

,ids,prediction,pos,wt,mut
5299,S42Y,0.310387,42,S,Y
6167,S42F,0.377899,42,S,F
10096,S42E,0.804154,42,S,E
9783,S42A,0.679917,42,S,A
7116,S42N,0.448137,42,S,N
...,...,...,...,...,...
9489,M533G,0.635888,533,M,G
9190,M533R,0.605504,533,M,R
9882,M533A,0.701457,533,M,A
8277,M533N,0.531492,533,M,N


In [67]:
# Load our file from the MPNN analysis

mio = pd.read_csv(MPNN_RAW)

In [68]:
# Reset the right sequence starting position to match the ESM_rank file

mio['Position'] = mio['Position'].apply(lambda x: x + OFFSET)

In [69]:
mio

,Mutant,WT_pos,Position,Mut_pos,Score
0,ARSB_A411,A,411,P,0.9199
1,ARSB_A229,E,229,G,0.8998
2,ARSB_A71,H,71,N,0.8674
3,ARSB_A430,H,430,T,0.8394
4,ARSB_A122,V,122,I,0.8320
...,...,...,...,...,...
212,ARSB_A402,S,402,T,0.1322
213,ARSB_A460,S,460,K,0.1319
214,ARSB_A428,S,428,K,0.1266
215,ARSB_A42,S,42,M,0.1186


In [70]:
riccardo

,ids,prediction,pos,wt,mut
5299,S42Y,0.310387,42,S,Y
6167,S42F,0.377899,42,S,F
10096,S42E,0.804154,42,S,E
9783,S42A,0.679917,42,S,A
7116,S42N,0.448137,42,S,N
...,...,...,...,...,...
9489,M533G,0.635888,533,M,G
9190,M533R,0.605504,533,M,R
9882,M533A,0.701457,533,M,A
8277,M533N,0.531492,533,M,N


In [71]:
# Add columns for position, wildtype, and mutation to the MPNN dataframe
mio['wt_pos_mut'] = mio['WT_pos'] + mio['Position'].astype(str) + mio['Mut_pos']

In [72]:
# Create a new column in the MPNN dataframe that matches the ESM_rank ids
mio['ESM_rank'] = mio['wt_pos_mut'].map(riccardo.set_index('ids')['prediction'])

In [73]:
# Remove the temporary column used for matching
mio.drop(columns=['wt_pos_mut'] , inplace=True)

In [74]:
mio

,Mutant,WT_pos,Position,Mut_pos,Score,ESM_rank
0,ARSB_A411,A,411,P,0.9199,0.629765
1,ARSB_A229,E,229,G,0.8998,0.519383
2,ARSB_A71,H,71,N,0.8674,0.650224
3,ARSB_A430,H,430,T,0.8394,0.387505
4,ARSB_A122,V,122,I,0.8320,0.636662
...,...,...,...,...,...,...
212,ARSB_A402,S,402,T,0.1322,0.716458
213,ARSB_A460,S,460,K,0.1319,0.558079
214,ARSB_A428,S,428,K,0.1266,0.504145
215,ARSB_A42,S,42,M,0.1186,0.413663


In [75]:
# remove positions considered detrimental by ESM_rank
mio = mio[~((mio['Mut_pos'] == 'A') & (mio['ESM_rank'] < 0.2))]

In [77]:
mio

,Mutant,WT_pos,Position,Mut_pos,Score,ESM_rank
0,ARSB_A411,A,411,P,0.9199,0.629765
1,ARSB_A229,E,229,G,0.8998,0.519383
2,ARSB_A71,H,71,N,0.8674,0.650224
3,ARSB_A430,H,430,T,0.8394,0.387505
4,ARSB_A122,V,122,I,0.8320,0.636662
...,...,...,...,...,...,...
212,ARSB_A402,S,402,T,0.1322,0.716458
213,ARSB_A460,S,460,K,0.1319,0.558079
214,ARSB_A428,S,428,K,0.1266,0.504145
215,ARSB_A42,S,42,M,0.1186,0.413663


In [78]:
# Add rank columns for Score and ESM_rank in the MPNN dataframe
mio[['rank_score' , 'rank_ESM']] = mio[['Score', 'ESM_rank']].rank(ascending=False , method='min')

In [79]:
# Calculate the average rank for each mutation in the MPNN dataframe    
# This will be used to sort the mutations based on their combined rank

mio['rank_avg'] = mio[['rank_score', 'rank_ESM']].mean(axis=1)

mio.sort_values(by='rank_avg', inplace=True)

mio.head()

,Mutant,WT_pos,Position,Mut_pos,Score,ESM_rank,rank_score,rank_ESM,rank_avg
25,ARSB_A214,I,214,L,0.6053,0.753723,26.0,12.0,19.0
16,ARSB_A429,V,429,I,0.6780,0.730400,17.0,23.0,20.0
27,ARSB_A177,S,177,T,0.5866,0.752594,28.0,13.0,20.5
40,ARSB_A472,L,472,I,0.5368,0.781434,41.0,7.0,24.0
12,ARSB_A222,L,222,I,0.7170,0.697913,13.0,35.0,24.0


In [80]:
mio.to_csv(OUTPUT_FILE , index=False)

In [81]:
mio['wt_pos_mut'] = mio['WT_pos'] + mio['Position'].astype(str) + mio['Mut_pos']

In [82]:
mutlist = mio['wt_pos_mut'].tolist()

In [84]:
with open('mutlist.txt' , 'w') as f:
    for mut in mutlist:
        f.write(mut + ' ')